In [1]:
from __future__ import print_function
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.utils.data
import torch.nn.init as init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import VAE

print( 'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

torch: 1.3.0+cpu cuda: False


## Load & Re-Scale Data Sets

In [2]:
path = "train_2_2000Samples.csv"
train_data = pd.read_csv(path).iloc[:,1:]
test_data = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_400Samples.csv')).iloc[:,1:]
train_data = train_data.iloc[:,:-1]
test_data = test_data.iloc[:,:-1]
train_data.columns  = test_data.columns
cols = test_data.columns
scalar = MinMaxScaler().fit(pd.concat([train_data, test_data]))
train_data = pd.DataFrame(scalar.transform (train_data)) 
test_data = pd.DataFrame(scalar.transform (test_data)) 
train_data.columns = cols
test_data.columns = cols

## Fix the Global Variables for Learning

In [3]:
#hyperparameters
x_dim = train_data.shape[1]
h_dim = 300
z_dim = 50
n_epochs = 100
clip = 10
learning_rate = 1e-3
batch_size = 10
seed = 100
print_every = 10
save_every = 10
#manual seed
torch.manual_seed(seed)
#init model + optimizer + datasets
train_loader = torch.utils.data.DataLoader ( dataset = train_data.values ,  batch_size = batch_size , shuffle= True)
test_loader = torch.utils.data.DataLoader (  dataset = test_data.values , shuffle= True)
model = VAE.VAE(x_dim, h_dim, z_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train and Test the VAE

In [4]:
def train(epoch):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_data) / batch_size ))
    epoch_div = np.zeros(int(len (train_data) / batch_size))
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        kld_loss, nll_loss, lat, recon, _ = model(data)
        epoch_loss [batch_idx] = nll_loss
        epoch_div [batch_idx] = kld_loss
        loss = kld_loss + nll_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, epoch_div
    
def test(epoch):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_kld_loss, mean_nll_loss = 0, 0
    epoch_loss = np.zeros(len(test_data))
    epoch_div = np.zeros(len(test_data))
    for i, (data) in enumerate(test_loader):                                           
        
        data = Variable(data.reshape(1,-1))
        kld_loss, nll_loss, _, _, _ = model(data)
        epoch_div [i] = kld_loss
        epoch_loss [i] = nll_loss
        mean_kld_loss += kld_loss.data
        mean_nll_loss += nll_loss.data

    mean_kld_loss /= len(test_loader.dataset)
    mean_nll_loss /= len(test_loader.dataset)

    print('====> Test set loss: KLD Loss = {:.4f}, NLL Loss = {:.4f} '.format(
        mean_kld_loss, mean_nll_loss))
    return epoch_loss, epoch_div

## Train and Test the Model

In [5]:
train_error = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
train_div = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
test_error , test_div  = np.zeros([n_epochs , test_data.shape[0]]) , np.zeros([n_epochs , test_data.shape[0]]) 
for epoch in range(1, n_epochs + 1):
    #training + testing
    tr = train(epoch)
    train_error [epoch-1 , :] = tr [0]
    train_div [epoch-1 , :] = tr [1] 
    te = test(epoch)
    test_error [epoch-1 , :] = te [0]
    test_div [epoch-1 , :] = te [1]

Train Epoch: 1 [0/2000 (0%)]	 KLD Loss: 19.267097 	 NLL Loss: 113.350119
Train Epoch: 1 [100/2000 (5%)]	 KLD Loss: 15.891589 	 NLL Loss: 69.577974
Train Epoch: 1 [200/2000 (10%)]	 KLD Loss: 11.400610 	 NLL Loss: 29.504055
Train Epoch: 1 [300/2000 (15%)]	 KLD Loss: 6.283135 	 NLL Loss: 27.296424
Train Epoch: 1 [400/2000 (20%)]	 KLD Loss: 3.735869 	 NLL Loss: 21.859380
Train Epoch: 1 [500/2000 (25%)]	 KLD Loss: 2.628333 	 NLL Loss: 19.601142
Train Epoch: 1 [600/2000 (30%)]	 KLD Loss: 2.472125 	 NLL Loss: 21.106033
Train Epoch: 1 [700/2000 (35%)]	 KLD Loss: 2.233010 	 NLL Loss: 18.474614
Train Epoch: 1 [800/2000 (40%)]	 KLD Loss: 2.477678 	 NLL Loss: 22.477546
Train Epoch: 1 [900/2000 (45%)]	 KLD Loss: 1.770213 	 NLL Loss: 18.684995
Train Epoch: 1 [1000/2000 (50%)]	 KLD Loss: 1.575751 	 NLL Loss: 18.720827
Train Epoch: 1 [1100/2000 (55%)]	 KLD Loss: 1.597024 	 NLL Loss: 17.986435
Train Epoch: 1 [1200/2000 (60%)]	 KLD Loss: 1.472890 	 NLL Loss: 19.808542
Train Epoch: 1 [1300/2000 (65%)]	 K

Train Epoch: 11 [400/2000 (20%)]	 KLD Loss: 2.057182 	 NLL Loss: -59.676628
Train Epoch: 11 [500/2000 (25%)]	 KLD Loss: 2.093826 	 NLL Loss: -60.831470
Train Epoch: 11 [600/2000 (30%)]	 KLD Loss: 2.273701 	 NLL Loss: -66.282122
Train Epoch: 11 [700/2000 (35%)]	 KLD Loss: 2.209313 	 NLL Loss: -58.102246
Train Epoch: 11 [800/2000 (40%)]	 KLD Loss: 2.158508 	 NLL Loss: -60.269228
Train Epoch: 11 [900/2000 (45%)]	 KLD Loss: 2.216939 	 NLL Loss: -55.766294
Train Epoch: 11 [1000/2000 (50%)]	 KLD Loss: 2.170945 	 NLL Loss: -52.343855
Train Epoch: 11 [1100/2000 (55%)]	 KLD Loss: 2.414125 	 NLL Loss: -64.224363
Train Epoch: 11 [1200/2000 (60%)]	 KLD Loss: 2.066354 	 NLL Loss: -45.977891
Train Epoch: 11 [1300/2000 (65%)]	 KLD Loss: 2.207417 	 NLL Loss: -51.136708
Train Epoch: 11 [1400/2000 (70%)]	 KLD Loss: 2.168362 	 NLL Loss: -58.235917
Train Epoch: 11 [1500/2000 (75%)]	 KLD Loss: 2.186500 	 NLL Loss: -55.891633
Train Epoch: 11 [1600/2000 (80%)]	 KLD Loss: 2.183632 	 NLL Loss: -55.220946
Train

Train Epoch: 21 [400/2000 (20%)]	 KLD Loss: 2.108155 	 NLL Loss: -80.583957
Train Epoch: 21 [500/2000 (25%)]	 KLD Loss: 2.160677 	 NLL Loss: -74.709092
Train Epoch: 21 [600/2000 (30%)]	 KLD Loss: 2.217537 	 NLL Loss: -74.908298
Train Epoch: 21 [700/2000 (35%)]	 KLD Loss: 2.243550 	 NLL Loss: -73.674773
Train Epoch: 21 [800/2000 (40%)]	 KLD Loss: 2.044946 	 NLL Loss: -77.793244
Train Epoch: 21 [900/2000 (45%)]	 KLD Loss: 2.272442 	 NLL Loss: -82.269278
Train Epoch: 21 [1000/2000 (50%)]	 KLD Loss: 2.225657 	 NLL Loss: -79.141559
Train Epoch: 21 [1100/2000 (55%)]	 KLD Loss: 2.128150 	 NLL Loss: -78.547892
Train Epoch: 21 [1200/2000 (60%)]	 KLD Loss: 2.041265 	 NLL Loss: -78.836287
Train Epoch: 21 [1300/2000 (65%)]	 KLD Loss: 2.279542 	 NLL Loss: -74.333741
Train Epoch: 21 [1400/2000 (70%)]	 KLD Loss: 2.198481 	 NLL Loss: -81.544578
Train Epoch: 21 [1500/2000 (75%)]	 KLD Loss: 2.285753 	 NLL Loss: -82.692317
Train Epoch: 21 [1600/2000 (80%)]	 KLD Loss: 2.199571 	 NLL Loss: -80.114574
Train

Train Epoch: 31 [400/2000 (20%)]	 KLD Loss: 1.988065 	 NLL Loss: -79.782227
Train Epoch: 31 [500/2000 (25%)]	 KLD Loss: 2.144032 	 NLL Loss: -79.037344
Train Epoch: 31 [600/2000 (30%)]	 KLD Loss: 1.921821 	 NLL Loss: -86.837609
Train Epoch: 31 [700/2000 (35%)]	 KLD Loss: 1.937395 	 NLL Loss: -80.100372
Train Epoch: 31 [800/2000 (40%)]	 KLD Loss: 2.139861 	 NLL Loss: -78.145173
Train Epoch: 31 [900/2000 (45%)]	 KLD Loss: 1.972460 	 NLL Loss: -69.393575
Train Epoch: 31 [1000/2000 (50%)]	 KLD Loss: 2.107969 	 NLL Loss: -76.473212
Train Epoch: 31 [1100/2000 (55%)]	 KLD Loss: 2.067514 	 NLL Loss: -83.028008
Train Epoch: 31 [1200/2000 (60%)]	 KLD Loss: 2.135100 	 NLL Loss: -79.766790
Train Epoch: 31 [1300/2000 (65%)]	 KLD Loss: 2.168240 	 NLL Loss: -77.951601
Train Epoch: 31 [1400/2000 (70%)]	 KLD Loss: 2.130019 	 NLL Loss: -84.799897
Train Epoch: 31 [1500/2000 (75%)]	 KLD Loss: 2.052233 	 NLL Loss: -82.209967
Train Epoch: 31 [1600/2000 (80%)]	 KLD Loss: 1.996636 	 NLL Loss: -85.724836
Train

Train Epoch: 41 [400/2000 (20%)]	 KLD Loss: 1.937918 	 NLL Loss: -87.143797
Train Epoch: 41 [500/2000 (25%)]	 KLD Loss: 1.984405 	 NLL Loss: -83.451218
Train Epoch: 41 [600/2000 (30%)]	 KLD Loss: 2.083981 	 NLL Loss: -82.548923
Train Epoch: 41 [700/2000 (35%)]	 KLD Loss: 1.961021 	 NLL Loss: -91.370722
Train Epoch: 41 [800/2000 (40%)]	 KLD Loss: 2.114208 	 NLL Loss: -85.991618
Train Epoch: 41 [900/2000 (45%)]	 KLD Loss: 2.020154 	 NLL Loss: -82.753172
Train Epoch: 41 [1000/2000 (50%)]	 KLD Loss: 2.040961 	 NLL Loss: -85.150375
Train Epoch: 41 [1100/2000 (55%)]	 KLD Loss: 2.113818 	 NLL Loss: -85.982134
Train Epoch: 41 [1200/2000 (60%)]	 KLD Loss: 2.083610 	 NLL Loss: -84.145151
Train Epoch: 41 [1300/2000 (65%)]	 KLD Loss: 1.845025 	 NLL Loss: -86.385703
Train Epoch: 41 [1400/2000 (70%)]	 KLD Loss: 1.920788 	 NLL Loss: -83.221966
Train Epoch: 41 [1500/2000 (75%)]	 KLD Loss: 2.053821 	 NLL Loss: -88.988217
Train Epoch: 41 [1600/2000 (80%)]	 KLD Loss: 1.968754 	 NLL Loss: -95.430878
Train

Train Epoch: 51 [400/2000 (20%)]	 KLD Loss: 2.120488 	 NLL Loss: -89.151295
Train Epoch: 51 [500/2000 (25%)]	 KLD Loss: 1.964986 	 NLL Loss: -85.596255
Train Epoch: 51 [600/2000 (30%)]	 KLD Loss: 2.022649 	 NLL Loss: -90.828198
Train Epoch: 51 [700/2000 (35%)]	 KLD Loss: 2.077977 	 NLL Loss: -98.834129
Train Epoch: 51 [800/2000 (40%)]	 KLD Loss: 2.051130 	 NLL Loss: -93.310473
Train Epoch: 51 [900/2000 (45%)]	 KLD Loss: 2.096624 	 NLL Loss: -96.004736
Train Epoch: 51 [1000/2000 (50%)]	 KLD Loss: 2.106781 	 NLL Loss: -93.917599
Train Epoch: 51 [1100/2000 (55%)]	 KLD Loss: 2.111601 	 NLL Loss: -98.666648
Train Epoch: 51 [1200/2000 (60%)]	 KLD Loss: 2.058536 	 NLL Loss: -76.560717
Train Epoch: 51 [1300/2000 (65%)]	 KLD Loss: 1.932451 	 NLL Loss: -88.596853
Train Epoch: 51 [1400/2000 (70%)]	 KLD Loss: 1.990745 	 NLL Loss: -94.434883
Train Epoch: 51 [1500/2000 (75%)]	 KLD Loss: 2.151588 	 NLL Loss: -96.481218
Train Epoch: 51 [1600/2000 (80%)]	 KLD Loss: 2.043401 	 NLL Loss: -84.403689
Train

Train Epoch: 61 [400/2000 (20%)]	 KLD Loss: 1.889029 	 NLL Loss: -90.989855
Train Epoch: 61 [500/2000 (25%)]	 KLD Loss: 2.133233 	 NLL Loss: -93.285107
Train Epoch: 61 [600/2000 (30%)]	 KLD Loss: 1.886009 	 NLL Loss: -94.427885
Train Epoch: 61 [700/2000 (35%)]	 KLD Loss: 2.217914 	 NLL Loss: -96.621244
Train Epoch: 61 [800/2000 (40%)]	 KLD Loss: 2.094527 	 NLL Loss: -85.759654
Train Epoch: 61 [900/2000 (45%)]	 KLD Loss: 2.069994 	 NLL Loss: -97.442513
Train Epoch: 61 [1000/2000 (50%)]	 KLD Loss: 2.142916 	 NLL Loss: -92.030613
Train Epoch: 61 [1100/2000 (55%)]	 KLD Loss: 2.046741 	 NLL Loss: -99.152065
Train Epoch: 61 [1200/2000 (60%)]	 KLD Loss: 2.105515 	 NLL Loss: -93.060908
Train Epoch: 61 [1300/2000 (65%)]	 KLD Loss: 2.023014 	 NLL Loss: -93.717965
Train Epoch: 61 [1400/2000 (70%)]	 KLD Loss: 2.104932 	 NLL Loss: -89.049901
Train Epoch: 61 [1500/2000 (75%)]	 KLD Loss: 2.018831 	 NLL Loss: -96.738361
Train Epoch: 61 [1600/2000 (80%)]	 KLD Loss: 1.994067 	 NLL Loss: -94.122393
Train

Train Epoch: 71 [400/2000 (20%)]	 KLD Loss: 2.205477 	 NLL Loss: -100.279190
Train Epoch: 71 [500/2000 (25%)]	 KLD Loss: 2.182680 	 NLL Loss: -96.391311
Train Epoch: 71 [600/2000 (30%)]	 KLD Loss: 2.131921 	 NLL Loss: -98.080440
Train Epoch: 71 [700/2000 (35%)]	 KLD Loss: 2.225259 	 NLL Loss: -96.395368
Train Epoch: 71 [800/2000 (40%)]	 KLD Loss: 2.246416 	 NLL Loss: -98.762449
Train Epoch: 71 [900/2000 (45%)]	 KLD Loss: 2.132813 	 NLL Loss: -96.164093
Train Epoch: 71 [1000/2000 (50%)]	 KLD Loss: 2.096465 	 NLL Loss: -98.847891
Train Epoch: 71 [1100/2000 (55%)]	 KLD Loss: 2.185107 	 NLL Loss: -91.766539
Train Epoch: 71 [1200/2000 (60%)]	 KLD Loss: 2.035087 	 NLL Loss: -101.681097
Train Epoch: 71 [1300/2000 (65%)]	 KLD Loss: 2.181104 	 NLL Loss: -100.421757
Train Epoch: 71 [1400/2000 (70%)]	 KLD Loss: 2.113731 	 NLL Loss: -90.730605
Train Epoch: 71 [1500/2000 (75%)]	 KLD Loss: 2.053729 	 NLL Loss: -92.830735
Train Epoch: 71 [1600/2000 (80%)]	 KLD Loss: 2.143613 	 NLL Loss: -97.587806
Tr

Train Epoch: 81 [400/2000 (20%)]	 KLD Loss: 2.348485 	 NLL Loss: -92.943851
Train Epoch: 81 [500/2000 (25%)]	 KLD Loss: 2.114322 	 NLL Loss: -95.490224
Train Epoch: 81 [600/2000 (30%)]	 KLD Loss: 2.141948 	 NLL Loss: -98.382737
Train Epoch: 81 [700/2000 (35%)]	 KLD Loss: 2.203362 	 NLL Loss: -89.031815
Train Epoch: 81 [800/2000 (40%)]	 KLD Loss: 2.379483 	 NLL Loss: -98.653955
Train Epoch: 81 [900/2000 (45%)]	 KLD Loss: 2.271662 	 NLL Loss: -92.214779
Train Epoch: 81 [1000/2000 (50%)]	 KLD Loss: 2.094141 	 NLL Loss: -97.221764
Train Epoch: 81 [1100/2000 (55%)]	 KLD Loss: 2.328393 	 NLL Loss: -93.004933
Train Epoch: 81 [1200/2000 (60%)]	 KLD Loss: 2.120554 	 NLL Loss: -99.329283
Train Epoch: 81 [1300/2000 (65%)]	 KLD Loss: 2.123713 	 NLL Loss: -91.488311
Train Epoch: 81 [1400/2000 (70%)]	 KLD Loss: 2.190107 	 NLL Loss: -96.703078
Train Epoch: 81 [1500/2000 (75%)]	 KLD Loss: 2.119811 	 NLL Loss: -90.814119
Train Epoch: 81 [1600/2000 (80%)]	 KLD Loss: 2.299157 	 NLL Loss: -95.021463
Train

Train Epoch: 91 [400/2000 (20%)]	 KLD Loss: 2.199690 	 NLL Loss: -102.807093
Train Epoch: 91 [500/2000 (25%)]	 KLD Loss: 2.157056 	 NLL Loss: -101.690613
Train Epoch: 91 [600/2000 (30%)]	 KLD Loss: 2.152946 	 NLL Loss: -92.110427
Train Epoch: 91 [700/2000 (35%)]	 KLD Loss: 2.182683 	 NLL Loss: -97.966515
Train Epoch: 91 [800/2000 (40%)]	 KLD Loss: 2.076619 	 NLL Loss: -92.301067
Train Epoch: 91 [900/2000 (45%)]	 KLD Loss: 2.161055 	 NLL Loss: -99.256045
Train Epoch: 91 [1000/2000 (50%)]	 KLD Loss: 2.207126 	 NLL Loss: -98.105494
Train Epoch: 91 [1100/2000 (55%)]	 KLD Loss: 2.330928 	 NLL Loss: -93.887822
Train Epoch: 91 [1200/2000 (60%)]	 KLD Loss: 2.169548 	 NLL Loss: -99.618445
Train Epoch: 91 [1300/2000 (65%)]	 KLD Loss: 2.206361 	 NLL Loss: -94.417520
Train Epoch: 91 [1400/2000 (70%)]	 KLD Loss: 2.347834 	 NLL Loss: -100.280132
Train Epoch: 91 [1500/2000 (75%)]	 KLD Loss: 2.237923 	 NLL Loss: -99.964839
Train Epoch: 91 [1600/2000 (80%)]	 KLD Loss: 2.348446 	 NLL Loss: -99.926378
Tr

## Get Latent Representations and Save the Model for later Use

In [6]:
train_lat = [ model (Variable(torch.tensor(train_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(train_data)) ]
test_lat = [ model (Variable(torch.tensor(test_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(test_data)) ]
train_lat = pd.DataFrame(torch.cat(train_lat).cpu().detach().numpy())
test_lat = pd.DataFrame(torch.cat(test_lat).cpu().detach().numpy())
cols = []
for i in range(train_lat.shape[1]):
    cols.append(str('Z'+str(i+1)))
train_lat.columns = cols
test_lat.columns = cols
train_lat.to_csv('Training_Data_Sets\\latent_100D.csv')
test_lat.to_csv('Test_Data_Sets\\latent_100D.csv')
with open('Models\\100D.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
array = np.zeros((10000,50))
for i in range(len(array)):
    array [i] = model(Variable(torch.tensor(np.zeros(100))))[-1].cpu().detach().numpy()
array1 = np.zeros((10000,50))
for i in range(len(array)):
    array1 [i] = model(Variable(torch.tensor(np.ones(100))))[-1].cpu().detach().numpy()

In [8]:
np.mean(abs(np.mean(array1, 0)-np.mean(array, 0)))/4

0.2689433305566933